In [1]:
import numpy as np
import pickle
import itertools
import pandas as pd

Формируем бинарный вектор длины Nx + N6a + N4a, где первое слагаемое отвечает за набор переменных, отражающих имеется ли заданное занятие в определенном временном слоте у определенной группы, а второе и третье - за дополнительные переменные для реализации ограничений в форме неравенств (как в задаче рюкзака)

In [2]:
Nt, Ng, Nw, Nd, Nh = 5, 2, 2, 6, 8
Ntg = Nt*Ng
T = Nw*Nd*Nh
Nx = T*Ntg

def v(tg, w, d, h):
    return T*tg + (w * Nd + d)*Nh + h

N6 = 7
Nwdg = Nw*Nd*Ng
N6a = N6*Nwdg

N4 = 5
Nwdt = Nw*Nd*Nt
N4a = N4*Nwdt

Формируем матрицу QUBO на основе ограничений задачи. Важно отметить, что мы не различаем пока предметы, которые ведет преподаватель, и, таким образом, у одной группы в день может быть вплоть до 4 занятий с одним и тем же преподавателем. Такой подход позволяет вдвое сократить число переменных Nx, а конкретные названия предметов несложно восстановить классически

In [3]:
Q = np.zeros((Nx + N6a + N4a, Nx + N6a + N4a))
offset = 0

# no simultaneous lessons in one group
P_LiG = 1
for (w,d,h, g) in itertools.product(range(Nw), range(Nd), range(Nh), range(Ng)):
    # sum(slots[n] for n in range(5)) <= 1  <=>  sum(slots[n1]*slots[n2] for perms(n1, n2)) 
    for (n1, n2) in itertools.permutations(range(Nt), 2):
        Q[v(g+Ng*n1,w,d,h),v(g+Ng*n2,w,d,h)] += P_LiG * 0.5

# no simultaneuos lessons with one teacher
P_LoT = 1
for (w,d,h, n) in itertools.product(range(Nw), range(Nd), range(Nh), range(Nt)):
    # sum(slots[g] for n in range(2)) <= 1  <=>  sum(slots[g1]*slots[g2] for perms(g1, g2)) 
    for (g1, g2) in itertools.permutations(range(Ng), 2):
        Q[v(g1+Ng*n,w,d,h),v(g2+Ng*n,w,d,h)] += P_LoT * 0.5
        
# no more that 6 lecs a day
P_nm6 = 1
for g in range(Ng):
    for (w, d) in itertools.product(range(Nw), range(Nd)):
        offset += P_nm6
        for i in range(N6):
            Q[Nx + N6*(g*Nw*Nd + w*Nd + d) + i, Nx + N6*(g*Nw*Nd + w*Nd + d) + i] += -2*P_nm6
            for j in range(N6):
                Q[Nx + N6*(g*Nw*Nd + w*Nd + d) + i, Nx + N6*(g*Nw*Nd + w*Nd + d) + j] += P_nm6
        xinds = [v(Ng*n+g,w,d,h) for n in range(Nt) for h in range(Nh)]
        inds = [*xinds, *list(range(Nx + N6*(g*Nw*Nd + w*Nd + d), Nx + N6*(g*Nw*Nd + w*Nd + d) + N6))]
        mults = np.concatenate((np.ones((Nt*Nh,)), -np.arange(N6)))
        for i1,m1 in zip(inds, mults):
            for i2,m2 in zip(inds, mults):
                Q[i1, i2] += P_nm6 * m1*m2
        
# less than 4 lectures on each teacher
P_nm4 = 1
for (w, d, n) in itertools.product(range(Nw), range(Nd), range(Nt)):
    offset += P_nm4
    for i in range(N4):
        Q[Nx+N6a + N4*(n*Nw*Nd + w*Nd + d) + i, Nx+N6a + N4*(n*Nw*Nd + w*Nd + d) + i] += -2*P_nm4
        for j in range(N4):
            Q[Nx+N6a + N4*(n*Nw*Nd + w*Nd + d) + i, Nx+N6a + N4*(n*Nw*Nd + w*Nd + d) + j] += P_nm4
    xinds = [v(Ng*n+g,w,d,h) for g in range(Ng) for h in range(Nh)]
    inds = [*xinds, *list(range(Nx+N6a + N4*(n*Nw*Nd + w*Nd + d), Nx+N6a + N4*(n*Nw*Nd + w*Nd + d) + N4))]
    mults = np.concatenate((np.ones((Ng*Nh,)), -np.arange(N4)))
    for i1,m1 in zip(inds, mults):
        for i2,m2 in zip(inds, mults):
            Q[i1, i2] += P_nm4 * m1*m2
        
# needed 8 lectures with one lector
P_L8 = 1
for (n,g) in itertools.product(range(Nt), range(Ng)):
    xinds = [v(Ng*n+g,w,d,h) for (w,d,h) in itertools.product(range(Nw), range(Nd), range(Nh))]
    offset += 64 * P_L8
    for i1 in xinds:
        Q[i1, i1] += -2 * 8 * P_L8
        for i2 in xinds:
            Q[i1, i2] += P_L8

# wishes of teachers
P_TW = 1
cants = [2, 0, 5, 1, 1]
for n, d in zip(range(Nt), cants):
    #for (g1, w1, h1, g2, w2, h2) in itertools.product(range(Ng), range(Nw), range(Nh), repeat=2):
    #    Q[v(g1+Ng*n,w1,d,h1),v(g2+Ng*n,w2,d,h2)] += P_TW
    for (g, w, h) in itertools.product(range(Ng), range(Nw), range(Nh)):
        Q[v(g+Ng*n,w,d,h),v(g+Ng*n,w,d,h)] += P_TW

In [4]:
with open('Q1.pkl','wb') as f:
     pickle.dump(Q, f)

In [5]:
import neal

In [6]:
variables = {i:i for i in range(Nx+N6+N4)}
linear = {i:Q[i,i] for i in range(Nx+N6+N4)}
quadratic = {(i,j):Q[i,j]+Q[j,i] for j in range(Nx+N6+N4) for i in range(j)}

Как вариант, можно получить решение с помощью локального солвера

In [7]:
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=100)
print(sampleset.first.energy)
from_dwave = True

-722.0


Или загрузить его из решения полученного на сервере с помощью SimCIMSolver

In [8]:
dat = []
with open("solution.yaml", 'r') as f:
    for i in range(7):
        f.readline()
    for i in range(Nx+N6a+N4a):
        s = f.readline()
        i = s.find(':')
        dat.append(round(float(s[i+1:])))

dat = np.array(dat, dtype=np.int32)
from_dwave = False

In [9]:
np.transpose(dat) @ Q @ dat

-720.0

In [10]:
def rev_dict(sd):
    return {value: key for key, value in sd.items()}

Формируем список всех занятий, а также восстанавливаем конкретные пары для каждого преподавателя. При этом в первую очередь обращаем внимание на дни, в которые у одной группы 3 и более занятия с ним, т.к. в таких ситуациях следует распределить 2 занятия на второй предмет, который ведет этот преподаватель

In [11]:
if from_dwave:
    dat = np.array(list(sampleset.first.sample.values()), dtype=np.int32)
shaped = np.reshape(dat[0:Nx], (Nt, Ng, Nw, Nd, Nh))

less_w_tutor = np.sum(shaped, axis=4)
inds = np.where(less_w_tutor >= 3)
changed = np.zeros(Ntg)
subjects = np.zeros(Nx, dtype=np.int32)
for i1, i2, i3, i4 in zip(*inds):
    # change exatly 2 subjects, can not be performed 3 such changes
    changed[i1*Ng + i2] += 2
    k = 0
    for h in range(Nh):
        if dat[v(Ng*i1+i2, i3, i4, h)]:
            subjects[v(Ng*i1+i2, i3, i4, h)] = 1
            k += 1
            if k > 1:
                break

entries = np.concatenate((np.transpose(np.array(np.where(shaped==1))), np.reshape(subjects[np.where(dat[0:Nx]==1)], (-1, 1)) ), axis=1)
for i in range(entries.shape[0]):
    if less_w_tutor[*entries[i,0:4]] < 3:
        if changed[entries[i, 0]*Ng + entries[i, 1]] < 4:
            changed[entries[i, 0]*Ng + entries[i, 1]] += 1
            entries[i,5] = 1

In [12]:
export = pd.DataFrame(columns=("Group", "Week", "Day", "Hour", "Subject", "Name"))
exp_dict = {"Group":[], "Week":[], "Day":[], "Hour":[], "Subject":[], "Name":[]}
custom_dict_group = rev_dict({'QC_1': 0, 'QC_2': 1})
custom_dict_week = rev_dict({'неделя 1': 0, 'неделя 2': 1})
custom_dict_day = rev_dict({'пн': 0, 'вт': 1, 'ср': 2, 'чт': 3, 'пт': 4, 'сб': 5, 'вс': 6})
custom_dict_hour = rev_dict({'9:00-10:00': 0, '10:00-11:00': 1, '11:00-12:00': 2, '12:00-13:00': 3, '13:00-14:00': 4, '14:00-15:00': 5, '15:00-16:00': 6, '16:00-17:00': 7})
custom_dict_subject = rev_dict({'Квантовая механика': 0, 'Квантовая теория информации': 1, 'Квантовые вычисления': 2, 'Сложность квантовых алгоритмов': 3, 'Квантовые алгоритмы в логистике': 4,
                        'Квантовое машинное обучение': 5, 'Моделирование квантовых систем': 6, 'Квантовые алгоритмы в химии': 7,'Физическая реализация квантовых компьютеров': 8,
                        'Моделирование квантовых алгоритмов': 9, 'нет занятий': 10})

custom_dict_teacher = rev_dict({'Иванов': 0, 'Петров': 1, 'Сидоров': 2, 'Карпов': 3, 'Соколов': 4, 'нет занятий': 10})


for entry in entries:
    exp_dict["Group"].append(custom_dict_group[entry[1]])
    exp_dict["Week"].append(custom_dict_week[entry[2]])
    exp_dict["Day"].append(custom_dict_day[entry[3]])
    exp_dict["Hour"].append(custom_dict_hour[entry[4]])
    exp_dict["Subject"].append(custom_dict_subject[2*entry[0]+entry[5]])
    exp_dict["Name"].append(custom_dict_teacher[entry[0]])
    

export = pd.DataFrame(exp_dict)
export.to_csv("solution.csv")
export.head(30)

,Group,Week,Day,Hour,Subject,Name
0,QC_1,неделя 1,пт,14:00-15:00,Квантовая теория информации,Иванов
1,QC_1,неделя 1,пт,16:00-17:00,Квантовая теория информации,Иванов
2,QC_1,неделя 1,сб,9:00-10:00,Квантовая теория информации,Иванов
3,QC_1,неделя 2,пн,15:00-16:00,Квантовая теория информации,Иванов
4,QC_1,неделя 2,вт,13:00-14:00,Квантовая механика,Иванов
5,QC_1,неделя 2,чт,9:00-10:00,Квантовая механика,Иванов
6,QC_1,неделя 2,пт,12:00-13:00,Квантовая механика,Иванов
7,QC_1,неделя 2,сб,16:00-17:00,Квантовая механика,Иванов
8,QC_2,неделя 1,пн,9:00-10:00,Квантовая теория информации,Иванов
9,QC_2,неделя 1,пн,10:00-11:00,Квантовая теория информации,Иванов


Контрольная проверка общего количества пар

In [13]:
len(export)

80

Файл Solution_proove реализует проверку, видно, что решение верное и удовлетворяет всем ограничениям. Однако, так получается не всегда. Оказывается, для минимального достигнутого значения функции -721.0 решение не удовлетворяет ограничению по дням, в которые работают преподаватели. Это свидетельствует о том, что имеется более глубокий локальный минимум, однако найти его не представляется возможным, да и не требуется, так как задача оказывается решенной уже при не самом низком значении функции QUBO